<p align="center">
  <img src="https://www.verifia.ca/assets/logo.png" width="160px" alt="VerifIA Logo"/><br>
  <strong>© 2025 VerifIA. All rights reserved.</strong>
</p>

In [ ]:
!pip install verifia[genflow]

### VerifIA - Rule-Based Verification Quickstart

This notebook demonstrates two approaches for generating a domain configuration for model verification using the California housing dataset. You can either manually create a simple domain dictionary based on your intuition or use VerifIA’s AI-powered generation flow to create a domain file automatically from the dataframe and descriptive domain knowledge.

#### 1. Importing Libraries and Setting Up

In this section, we import the required Python libraries and modules. These include standard data science package, scikit-learn, and the specific modules from the VerifIA tool that handle model wrapping and rule verification.

In [ ]:
%load_ext autoreload
%autoreload 2
import os
import getpass
from dotenv import load_dotenv
load_dotenv() 
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import root_mean_squared_error, mean_absolute_percentage_error
from sklearn.model_selection import train_test_split
from sklearn.datasets import fetch_california_housing
from verifia.models import SKLearnModel, build_from_model_card
from verifia.verification.results import RulesViolationResult
from verifia.verification.verifiers import RuleConsistencyVerifier

#### 2. Setting Constants and Directories

We define some key constants such as the random seed for reproducibility and the directory path where our model artifacts will be saved.

In [ ]:
RAND_SEED = 0
MODELS_DIRPATH = "../models"

#### 3. Loading the California Housing Data

Using scikit-learn’s `fetch_california_housing`, we load the California housing dataset into a DataFrame. We extract the feature names and target name directly from the dataset, which will be used later for model training and verification. The dataset is split into training and testing subsets (80/20 split) to allow for model training on one set and evaluation/verification on the other.

In [ ]:
housing = fetch_california_housing(as_frame=True)
housing_df = housing.frame
feature_names = housing.feature_names
target_name = housing.target_names[0]
train_df, test_df = train_test_split(housing_df, train_size=0.8, random_state=RAND_SEED)

#### 5. Training Multiple Regression Models

We create and train several regression models using different pipelines:
- **Polynomial Regression:** A pipeline with standard scaling, polynomial feature expansion (degree 2), and linear regression.
- **Decision Tree Regressor:** A simple decision tree model with standard scaling.
- **Random Forest Regressor:** An ensemble model using random forests.
- **MLP Regressor:** A multi-layer perceptron with a specific hidden layer configuration.

Each model is trained on the training data, and key performance metrics (RMSE and MAPE) are computed on the test set.

##### 5.1 Optional: Run this cell to train the polynomial regression model
This cell trains a polynomial regression model using a 2nd degree polynomial expansion. The model is scaled with StandardScaler and fit using LinearRegression, which helps capture non-linear relationships in your data.

In [ ]:
poly_reg = make_pipeline(StandardScaler(), PolynomialFeatures(degree=2), LinearRegression())
poly_reg.fit(train_df[feature_names], train_df[target_name])
poly_predictions = poly_reg.predict(test_df[feature_names])
poly_rmse = root_mean_squared_error(test_df[target_name], poly_predictions)
poly_mape = mean_absolute_percentage_error(test_df[target_name], poly_predictions)
print(poly_rmse, poly_mape)

##### 5.2 Optional: Run this cell to train the decision tree regression model
This cell trains a decision tree regressor using StandardScaler and DecisionTreeRegressor. Decision trees are useful for capturing non-linear patterns without requiring explicit feature engineering.



In [ ]:
tree_reg = make_pipeline(StandardScaler(), DecisionTreeRegressor(random_state=RAND_SEED))
tree_reg.fit(train_df[feature_names], train_df[target_name])
tree_predictions = tree_reg.predict(test_df[feature_names])
tree_rmse = root_mean_squared_error(test_df[target_name], tree_predictions)
tree_mape = mean_absolute_percentage_error(test_df[target_name], tree_predictions)
print(tree_rmse, tree_mape)

##### 5.3 Optional: Run this cell to train the random forest regression model
This cell trains a random forest model, which aggregates multiple decision trees to improve robustness and accuracy. It uses StandardScaler along with RandomForestRegressor to capture ensemble learning benefits.

In [ ]:
forest_reg = make_pipeline(StandardScaler(), RandomForestRegressor(random_state=RAND_SEED))
forest_reg.fit(train_df[feature_names], train_df[target_name])
forest_predictions = forest_reg.predict(test_df[feature_names])
forest_rmse = root_mean_squared_error(test_df[target_name], forest_predictions)
forest_mape = mean_absolute_percentage_error(test_df[target_name], forest_predictions)
print(forest_rmse, forest_mape)

##### 5.4 Optional: Run this cell to train the neural network regression model

In [ ]:
mlp_reg = make_pipeline(StandardScaler(), 
                        MLPRegressor(hidden_layer_sizes=(128,64,32), activation='relu', solver='adam', random_state=RAND_SEED))
mlp_reg.fit(train_df[feature_names], train_df[target_name])
test_predictions = mlp_reg.predict(test_df[feature_names])
mlp_rmse = root_mean_squared_error(test_df[target_name], test_predictions)
mlp_mape = mean_absolute_percentage_error(test_df[target_name], test_predictions)
print(mlp_rmse, poly_mape)

#### 6. Wrapping the Model with VerifIA

VerifIA requires that the model is wrapped in a standardized model wrapper. Here, we use the `build_from_model_card` function to create a `SKLearnModel` instance that encapsulates essential metadata (such as model name, version, type, feature names, target name, and storage directory) along with the trained model. 

**Note:** You can swap out the pipeline (e.g., use tree, forest, or mlp regressors) by adjusting the wrapper configuration and replacing the wrapped model object.


In [ ]:
model_wrapper:SKLearnModel = build_from_model_card({
    "name": "CHPrice_skl_poly_regressor", # instead of "poly", you should put "tree", "forest", "mlp"
    "version": "1",
    "type": "regression",
    "description": "model predicts the prices of california houses",
    "framework": "sklearn",
    "feature_names": feature_names,
    "target_name": target_name,
    "local_dirpath": MODELS_DIRPATH
}).wrap_model(poly_reg) # instead of poly_reg, you should put tree_reg, forest_reg, mlp_reg

#### 7. Creating the Domain Configuration

We present two options for creating the domain configuration:

##### **Option A – Manual Domain Dictionary**

You can manually create a simple domain dictionary. In this example, a dictionary is built where each variable is defined based on the features from the California housing dataframe. A sample constraint (e.g., a ratio between average bedrooms and rooms) and a rule (R1) are included. You can further customize and extend this dictionary with additional rules as needed.

In [ ]:
domain_cfg_dict = {
    "variables":{
        col: {
            "type": "INT" if (is_int := (housing_df[col] == housing_df[col].round()).all()) else "FLOAT",
            "range": (housing_df[col].astype(int) if is_int else housing_df[col]).agg(['min', 'max']).tolist()
        }
        for col in housing_df.columns
    },
    "constraints":{
        "C1": {
                "description":"", 
                "formula": "AveBedrms/AveRooms > 0.5"
            }
    },
    "rules":{
        "R1": {
               "description": "",
               "premises": {"AveRooms":"inc", "AveBedrms":"inc", "HouseAge": "dec"},
               "conclusion": {"MedHouseVal":"inc"}
            }
    }
}
domain_cfg_dict["variables"]['MedHouseVal']['insignificant_variation'] = 0.15 # expect 15% of error as acceptable

##### **Option B – AI-Powered Domain Generation**
Alternatively, you can leverage VerifIA’s `DomainGenFlow` to generate a domain dictionary automatically. By providing the dataframe and a description (here, the dataset’s description from `housing.DESCR`), the tool generates a domain configuration using AI. Additionally, you can customize the GPT configuration for domain dictionary generation by setting the environment variables `VERIFIA_GPT_NAME` and `VERIFIA_GPT_TEMPERATURE`. The defaults for these variables are: `VERIFIA_GPT_NAME` is set to "gpt-4o-mini" and `VERIFIA_GPT_TEMPERATURE` is set to 0. You can include them in a `.env` file along with your `OPEN_API_KEY` if you want to change the default options.

**Setup OpenAI and LangSmith Keys**

Note that LangSmith is not needed, but it is helpful. If you do want to use LangSmith, after you sign up at the link above, make sure to set your environment variables to start logging traces.

Accessing the OpenAI API requires an API key, which you can get by creating an account. Once you have a key you'll want to set it as an environment variable by running:

In [ ]:
os.environ["LANGCHAIN_TRACING_V2"] = 'true'
os.environ["LANGCHAIN_ENDPOINT"] = 'https://api.smith.langchain.com'
os.environ["LANGCHAIN_API_KEY"] = getpass.getpass(prompt='Your LANGCHAIN_API_KEY? ')
os.environ["OPENAI_API_KEY"] = getpass.getpass(prompt='Your OPENAI_API_KEY? ')
os.environ["USER_AGENT"] = 'my_agent'
os.environ["LANGCHAIN_PROJECT"] = 'VERIFIA_TEST'
os.environ["VERIFIA_GPT_NAME"] = 'gpt-4.1'

In [ ]:
from verifia.generation import DomainGenFlow

domain_genflow = DomainGenFlow()
domain_genflow.load_ctx(dataframe=housing_df, 
                        db_str_content=str(housing.DESCR),
                        model_card=model_wrapper.model_card.to_dict())
domain_cfg_dict = domain_genflow.run(save=True, local_path="./domain.yaml")

#### 8. Instantiating the Rule Consistency Verifier

Using the constructed or generated domain configuration, we instantiate a `RuleConsistencyVerifier`. This component is responsible for checking the consistency of the model with respect to the defined rules and constraints.

In [ ]:
model_verifier = RuleConsistencyVerifier(domain_cfg_dict)

#### 9. Configuring and Running the Verification

Next, we connect the verifier to our wrapped model and the test dataframe.

In [ ]:
model_verifier.verify(model_wrapper).on(test_df)

In this example, we choose to run the verification using a Genetic Algorithm (GA) as the search strategy. We specify parameters such as:
- **Population Size:** 50
- **Maximum Iterations:** 10
- **Original Seed Size:** 100

The verifier explores the input space to identify any rule violations.

In [ ]:
result:RulesViolationResult = model_verifier.using("RS")\
                                            .run(pop_size=50, max_iters=10, 
                                                 orig_seed_size=100)

#### 10. Saving the Results and Model Artifacts

Finally, the verification results are saved as an HTML report, which provides a comprehensive summary of the rule evaluations and any detected inconsistencies. Additionally, the model and its model card are saved for future reference and further analysis.

In [ ]:
result.save_as_html("../reports/CHPrice_skl_poly_regressor.html")  
# "poly" is used for the polynomial model, 
# ensure that the report's filename includes a short identifier reflecting the specific model used. 
# For instance, use "tree" for a decision tree model, "forest" for a random forest model, and "mlp" for a multi-layer perceptron. 
# This naming convention makes it easier to quickly identify which model generated the report.

In [ ]:
model_wrapper.save_model()
model_wrapper.save_model_card("../models/CHPrice_skl_poly_regressor.yaml")
# "poly" is used for the polynomial model, 
# ensure that the model card's filename includes a short identifier reflecting the specific model used. 
# For instance, use "tree" for a decision tree model, "forest" for a random forest model, and "mlp" for a multi-layer perceptron. 
# This naming convention makes it easier to quickly identify which model is described by the card.